In [6]:
import numpy as np
import matplotlib.pyplot as plt
from control import StateSpace, forced_response
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from ipywidgets import interact, VBox, HBox, Output, HTMLMath
from IPython.display import HTML

HTML("""
<style>
     
.widget-label, .custom-title {
    font-weight: bold !important;
    font-size: 16px !important;
}

.widget-toggle-buttons .widget-toggle-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
.widget-toggle-buttons .widget-toggle-button.selected {
    background-color: #007acc !important;
    color: white !important;
}

button.widget-toggle-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
button.widget-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}     
</style>
""")

In [7]:
class SystemSimulator:
    def __init__(self, A = None, B = None, C = None, D = None, nonlinear_func = None):
        self.sys_lin = None
        if A is not None and B is not None and C is not None and D is not None:
            self.sys_lin = StateSpace(A, B, C, D)
        self.nonlinear_func = nonlinear_func

    def simulate(self, t, x0, u_func):
        results = {}

        if self.sys_lin is not None:
            U = [u_func(tt) for tt in t]
            T_out, y_lin = forced_response(self.sys_lin, T=t, U=U, X0=x0)
            if y_lin.ndim > 1:
                results["linear"] = y_lin[0]  
            else:
                results["linear"] = y_lin
        
        if self.nonlinear_func is not None:
            sol = solve_ivp(lambda tau, x: self.nonlinear_func(tau, x, u_func), 
                            [t[0], t[-1]], x0, t_eval=t)
            results["nonlinear"] = sol.y[0]
        return results 

    def plot(self, t, results):
        plt.figure(figsize=(8,5))
        if "linear" in results:
            plt.plot(t, results["linear"], label = "Linear system")
        if "nonlinear" in results:
            plt.plot(t, results["nonlinear"], "--", label = "Non-linear system")

        plt.xlabel("Time [s]")
        plt.ylabel("Output y(t)")
        plt.title("System Response")
        plt.legend()
        plt.grid(True)
        plt.show()               

In [8]:
def parse_matrix(text):
    """
    Valid example: "[[0,1],[-2,-3]]" -> np.array([[0,1],[-2,-3]])
    """
    try:
        return np.array(eval(text))
    except:
        return None

def parse_nonlinear(expr_str):
    """
    Valid example:
    "[x[1], -2*x[0] - 3*x[1] + u + 0.2*(x[0]**3)]"
    """
    def dyn(t, x, u_func):
        u_val = u_func(t) 
        return eval(expr_str, {"t": t, "x": x, "u": u_val, "np": np})
    return dyn



In [9]:
style_bold = {'description_width': '150px'}
layout_box = widgets.Layout(width="300px", height="70px")

A_box = widgets.Textarea(
    value="[[0, 1], [-2, -3]]",
    description="Matrix A:",
    layout=layout_box,
    style={'description_width': 'initial'}
)
A_box.layout = widgets.Layout(width="300px", height="70px")
A_box.add_class("bold-label")

B_box = widgets.Textarea(
    value="[[0], [1]]",
    description="Matrix B:",
    layout=layout_box,
    style={'description_width': 'initial'}
)

C_box = widgets.Textarea(
    value="[[1, 0]]",
    description="Matrix C:",
    layout=layout_box,
    style={'description_width': 'initial'}
)

D_box = widgets.Textarea(
    value="[[0]]",
    description="Matrix D:",
    layout=layout_box,
    style={'description_width': 'initial'}
)

nonlin_box = widgets.Textarea(
    value="[x[1], -2*x[0] - 3*x[1] + u + 0.2*(x[0]**3)]",
    description="Non-linear f(x,u,t):",
    layout=widgets.Layout(width="620px", height="70px"),
    style={'description_width': 'initial'}
)

input_box = widgets.ToggleButtons(
    options=["Step", "Impulse", "Sine", "Custom"],
    value="Step",
    description="Input:",
    button_style='info',
)

# sliders
T_box = widgets.FloatSlider(value=10, min=0, max=100, step=1, description="T:")

init_label = widgets.HTML(value="<span class='custom-title'>Initial conditions:</span>")
time_label = widgets.HTML(value="<span class='custom-title'>Time:</span>")

x0_0_label = widgets.HTML(value="<b>x<sub>1</sub>(0):</b>")
x0_0_box = widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, layout=widgets.Layout(width="300px"))

x0_1_label = widgets.HTML(value="<b>x<sub>2</sub>(0):</b>")
x0_1_box = widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, layout=widgets.Layout(width="300px"))

x0_0_ui = HBox([x0_0_label, x0_0_box])
x0_1_ui = HBox([x0_1_label, x0_1_box])

# button for non-linear
show_nonlin_box = widgets.ToggleButton(
    value=True,
    description="Show non-linear",
    button_style='success',
    layout=widgets.Layout(width="200px", height="40px")
)

reset_button = widgets.Button(
    value=False,
    description="Reset",
    button_style='danger', 
    layout=widgets.Layout(width="200px", height="40px")
)

# final layout
ui = VBox([
    HBox([A_box, B_box]),
    HBox([C_box, D_box]),
    nonlin_box,
    input_box,
    init_label,
    HBox([x0_0_ui, x0_1_ui]),
    time_label,
    T_box,
    HBox([show_nonlin_box, reset_button])
])


In [ ]:
out = Output()

def on_change(change=None):
    out.clear_output()
    with out:
        A = parse_matrix(A_box.value)
        B = parse_matrix(B_box.value)
        C = parse_matrix(C_box.value)
        D = parse_matrix(D_box.value)

        nonlinear_func = None
        if nonlin_box.value.strip() != "":
            nonlinear_func = parse_nonlinear(nonlin_box.value)

        sim = SystemSimulator(A, B, C, D, nonlinear_func)

        # time and initial conditions
        t = np.linspace(0, T_box.value, 500)
        x0 = [x0_0_box.value, x0_1_box.value]

        # input
        if input_box.value == "Step":
            u_func = lambda t: 1.0 if t >= 0 else 0.0
        elif input_box.value == "Impulse":
            u_func = lambda t: 1.0 if abs(t) < 1e-2 else 0.0
        elif input_box.value == "Sine":
            u_func = lambda t: np.sin(2*np.pi*0.5*t)
        else:
            u_func = lambda t: np.exp(-0.5*t)*np.cos(2*t)

        # simulation
        results = sim.simulate(t, x0, u_func)
        if not show_nonlin_box.value and "nonlinear" in results:
            results.pop("nonlinear")

        # plot
        sim.plot(t, results)

def reset_values(b):
    # base matrixes
    A_box.value = "[[0, 1], [-2, -3]]"
    B_box.value = "[[0], [1]]"
    C_box.value = "[[1, 0]]"
    D_box.value = "[[0]]"

    # nonlinear function
    nonlin_box.value = "[x[1], -2*x[0] - 3*x[1] + u + 0.2*(x[0]**3)]"

    # Input
    input_box.value = "Step"

    # initial conditions
    x0_0_box.value = 0.0
    x0_1_box.value = 0.0

    # time
    T_box.value = 10.0

    # Toggle nonlinear
    show_nonlin_box.value = True

reset_button.on_click(reset_values)

for w in [A_box, B_box, C_box, D_box, nonlin_box,
          input_box, T_box, x0_0_box, x0_1_box, show_nonlin_box]:
    w.observe(on_change, names="value")

# show everything together
display(ui, out)
on_change() 


Output()